In [ ]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def plot_phase_coherence(number_simulation_runs, range_initial_states, coupling, network, a, b, tau, delta_t, T):
    '''
    number_simulation_runs: integer
        number of simulations to run for phase coherence plots
    
    range_initial_states: positiv integer
        size of rectangle around origin from which initial states are drawn according to uniform distribution
    
    coupling: float
        coupling strength between oscillators
    
    network: networkx graph
        
    a, b, tau: float
        parameters for FitzHugh-Nagumo
    
    delta_t, T: float
        for Euler approximation of the differential equations
          
          
    phase coherence measure: 
    "The synchronization of FitzHugh–Nagumo neuron network coupled by gap junction" - Zhan Yong, Zhang Su-Hua, Zhao Tong-Jun, An Hai-Long, Zhang Zhen-Dong, Han Ying-Rong, Liu Hui, and Zhang Yu-Hong
    
                     1
            R(t) =  --- * sum_i,j { [v_i(t) - v_j(t)]^2 } 
                    N^2
                    
        where {} denotes the average of a stochastic random variable
    '''
    # to generate random initial conditions for every node and every run
    number_nodes = G.number_of_nodes()
    
    # create matrix with dimensions (time steps in simulation x number of runs)
    # one simulation run measures the coherence of the whole network for all time steps
    # at the end one simulation run is one curve in the plot 
    phase_coherence_v = np.empty([int(T/delta_t)+1, number_simulation_runs], dtype=float)
    phase_coherence_w = np.empty([int(T/delta_t)+1, number_simulation_runs], dtype=float)

    # for every run we want different random initial states -> generate new for every run!
    for simulation_run in range(int(number_simulation_runs)): 
        model = FitzHugh_Nagumo_coupled(coupling=coupling, network=network, a=a, b=b, tau=tau, delta_t=delta_t, T=T, initial_states=np.array([np.random.uniform(-range_initial_states, range_initial_states, size=number_nodes), np.random.uniform(-range_initial_states, range_initial_states, size=number_nodes)]))
        act_mat = model.run(model.adj_mat) 
        t_steps, N = act_mat[0].shape
        
        for timestep in range(int(t_steps)):
            A0 = act_mat[0] # states of v
            A1 = act_mat[1] # states of w
            
            # get the difference of the states for all nodes for fixed time step 'timestep':
            # take empirical average since there is no random variable involved
            # because the difference is taken of two states divide by 2 to take average
            state_vi, state_vj = np.meshgrid(A0[timestep, :], A0[timestep, :])
            state_wi, state_wj = np.meshgrid(A1[timestep, :], A1[timestep, :])
            
            # absolute value because pairwise differences are squared in formula and therefore positive
            interactions_v = (1/2)*np.abs((state_vi - state_vj))
            interactions_w = (1/2)*np.abs((state_wi - state_wj))
            
            # to take the squares of the differences in the formula we multilply the array with its transposed
            # note that: entry A_ij in resulting matrix the sum over all j for fixed i
            intact_sq_v = np.matmul(interactions_v, interactions_v.T)
            intact_sq_w = np.matmul(interactions_w, interactions_w.T)
            
            # to get double sum in formula we only need to sum up all entries of the first column 
            coherence_v = N**(-2)*(intact_sq_v[:,0]).sum(axis=0) 
            coherence_w = N**(-2)*(intact_sq_w[:,0]).sum(axis=0)

            # write phase coherence into matrix for every timestep and run
            phase_coherence_v[timestep, simulation_run] = coherence_v
            phase_coherence_w[timestep, simulation_run] = coherence_w
     
    
    
    # now use phase_coherence_v/-w matrices to make plots
    _, ax = plt.subplots(figsize=(12, 4))
    ax.plot(phase_coherence_v)
    sns.set_style("whitegrid")
    plt.suptitle('simulations = %s' %number_simulation_runs)
    ax.set_xlabel('Time t', fontsize=25)
    ax.set_ylabel('R(t) for v', fontsize=25)
    
    _, ax = plt.subplots(figsize=(12, 4))
    ax.plot(phase_coherence_w)
    ax.set_xlabel('Time t', fontsize=25)
    ax.set_ylabel('R(t) for w', fontsize=25)
    
    return ax
        
        